# Table of Contents
 <p><div class="lev1 toc-item"><a href="#パッケージのimport" data-toc-modified-id="パッケージのimport-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>パッケージのimport</a></div><div class="lev1 toc-item"><a href="#動画の描画関数の宣言" data-toc-modified-id="動画の描画関数の宣言-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>動画の描画関数の宣言</a></div><div class="lev1 toc-item"><a href="#namedtuple" data-toc-modified-id="namedtuple-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>namedtuple</a></div><div class="lev1 toc-item"><a href="#定数の設定" data-toc-modified-id="定数の設定-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>定数の設定</a></div><div class="lev1 toc-item"><a href="#経験を保存するメモリクラスを定義します" data-toc-modified-id="経験を保存するメモリクラスを定義します-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>経験を保存するメモリクラスを定義します</a></div><div class="lev2 toc-item"><a href="#sandbox" data-toc-modified-id="sandbox-51"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>sandbox</a></div><div class="lev1 toc-item"><a href="#Brain" data-toc-modified-id="Brain-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Brain</a></div><div class="lev2 toc-item"><a href="#brain-解説（obsolete)" data-toc-modified-id="brain-解説（obsolete)-61"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>brain 解説（obsolete)</a></div><div class="lev3 toc-item"><a href="#initialization" data-toc-modified-id="initialization-611"><span class="toc-item-num">6.1.1&nbsp;&nbsp;</span>initialization</a></div><div class="lev3 toc-item"><a href="#replay" data-toc-modified-id="replay-612"><span class="toc-item-num">6.1.2&nbsp;&nbsp;</span>replay</a></div><div class="lev4 toc-item"><a href="#メモリサイズの確認" data-toc-modified-id="メモリサイズの確認-6121"><span class="toc-item-num">6.1.2.1&nbsp;&nbsp;</span>メモリサイズの確認</a></div><div class="lev4 toc-item"><a href="#ミニバッチの作成" data-toc-modified-id="ミニバッチの作成-6122"><span class="toc-item-num">6.1.2.2&nbsp;&nbsp;</span>ミニバッチの作成</a></div><div class="lev4 toc-item"><a href="#教師信号となるQ(s_t,-a_t)値を求める" data-toc-modified-id="教師信号となるQ(s_t,-a_t)値を求める-6123"><span class="toc-item-num">6.1.2.3&nbsp;&nbsp;</span>教師信号となるQ(s_t, a_t)値を求める</a></div><div class="lev2 toc-item"><a href="#解説終わり" data-toc-modified-id="解説終わり-62"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>解説終わり</a></div><div class="lev1 toc-item"><a href="#Agent" data-toc-modified-id="Agent-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Agent</a></div><div class="lev1 toc-item"><a href="#Environment" data-toc-modified-id="Environment-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Environment</a></div><div class="lev2 toc-item"><a href="#original" data-toc-modified-id="original-81"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>original</a></div><div class="lev2 toc-item"><a href="#多少改造したenvironment" data-toc-modified-id="多少改造したenvironment-82"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>多少改造したenvironment</a></div><div class="lev1 toc-item"><a href="#classでなくしてみる" data-toc-modified-id="classでなくしてみる-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>classでなくしてみる</a></div><div class="lev2 toc-item"><a href="#envのinit" data-toc-modified-id="envのinit-91"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>envのinit</a></div><div class="lev3 toc-item"><a href="#Agent作成" data-toc-modified-id="Agent作成-911"><span class="toc-item-num">9.1.1&nbsp;&nbsp;</span>Agent作成</a></div><div class="lev2 toc-item"><a href="#envのfor-loop" data-toc-modified-id="envのfor-loop-92"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>envのfor loop</a></div><div class="lev3 toc-item"><a href="#agent.get_actionはepsilon-greedy法により決定されたaction(この例の場合、右か左かのゼロイチ）を返す" data-toc-modified-id="agent.get_actionはepsilon-greedy法により決定されたaction(この例の場合、右か左かのゼロイチ）を返す-921"><span class="toc-item-num">9.2.1&nbsp;&nbsp;</span>agent.get_actionは$\epsilon$-greedy法により決定されたaction(この例の場合、右か左かのゼロイチ）を返す</a></div><div class="lev3 toc-item"><a href="#actionを与えるとenvがどうなるかを計算してくれる：" data-toc-modified-id="actionを与えるとenvがどうなるかを計算してくれる：-922"><span class="toc-item-num">9.2.2&nbsp;&nbsp;</span>actionを与えるとenvがどうなるかを計算してくれる：</a></div><div class="lev3 toc-item"><a href="#終了していたら" data-toc-modified-id="終了していたら-923"><span class="toc-item-num">9.2.3&nbsp;&nbsp;</span>終了していたら</a></div><div class="lev3 toc-item"><a href="#エージェントが（状態、アクション、次のアクション、報酬）を記憶" data-toc-modified-id="エージェントが（状態、アクション、次のアクション、報酬）を記憶-924"><span class="toc-item-num">9.2.4&nbsp;&nbsp;</span>エージェントが（状態、アクション、次のアクション、報酬）を記憶</a></div><div class="lev3 toc-item"><a href="#Experience-ReplayでQ関数を更新する" data-toc-modified-id="Experience-ReplayでQ関数を更新する-925"><span class="toc-item-num">9.2.5&nbsp;&nbsp;</span>Experience ReplayでQ関数を更新する</a></div><div class="lev2 toc-item"><a href="#brain" data-toc-modified-id="brain-93"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>brain</a></div><div class="lev3 toc-item"><a href="#サンプルがたまった学習済みのbrainを拝借" data-toc-modified-id="サンプルがたまった学習済みのbrainを拝借-931"><span class="toc-item-num">9.3.1&nbsp;&nbsp;</span>サンプルがたまった学習済みのbrainを拝借</a></div><div class="lev3 toc-item"><a href="#mini-batch取得" data-toc-modified-id="mini-batch取得-932"><span class="toc-item-num">9.3.2&nbsp;&nbsp;</span>mini batch取得</a></div><div class="lev3 toc-item"><a href="#mini-batchをtorchが食える形式に変形" data-toc-modified-id="mini-batchをtorchが食える形式に変形-933"><span class="toc-item-num">9.3.3&nbsp;&nbsp;</span>mini batchをtorchが食える形式に変形</a></div><div class="lev3 toc-item"><a href="#推論モードへ切り替え" data-toc-modified-id="推論モードへ切り替え-934"><span class="toc-item-num">9.3.4&nbsp;&nbsp;</span>推論モードへ切り替え</a></div><div class="lev3 toc-item"><a href="#ネットワークが出力したQ(s_t,-a_t)を求める-(state_action_valueに格納）" data-toc-modified-id="ネットワークが出力したQ(s_t,-a_t)を求める-(state_action_valueに格納）-935"><span class="toc-item-num">9.3.5&nbsp;&nbsp;</span>ネットワークが出力したQ(s_t, a_t)を求める (state_action_valueに格納）</a></div><div class="lev3 toc-item"><a href="#教師となる行動価値関数（expected_state_action_values）を計算" data-toc-modified-id="教師となる行動価値関数（expected_state_action_values）を計算-936"><span class="toc-item-num">9.3.6&nbsp;&nbsp;</span>教師となる行動価値関数（expected_state_action_values）を計算</a></div><div class="lev4 toc-item"><a href="#doneでないnext-state:" data-toc-modified-id="doneでないnext-state:-9361"><span class="toc-item-num">9.3.6.1&nbsp;&nbsp;</span>doneでないnext state:</a></div><div class="lev4 toc-item"><a href="#next-statenにおける行動価値関数" data-toc-modified-id="next-statenにおける行動価値関数-9362"><span class="toc-item-num">9.3.6.2&nbsp;&nbsp;</span>next statenにおける行動価値関数</a></div><div class="lev4 toc-item"><a href="#next-stateにおける行動価値関数の最大値" data-toc-modified-id="next-stateにおける行動価値関数の最大値-9363"><span class="toc-item-num">9.3.6.3&nbsp;&nbsp;</span>next stateにおける行動価値関数の最大値</a></div><div class="lev4 toc-item"><a href="#教師となるQ(s_t,-a_t)値を、Q学習の式から求める" data-toc-modified-id="教師となるQ(s_t,-a_t)値を、Q学習の式から求める-9364"><span class="toc-item-num">9.3.6.4&nbsp;&nbsp;</span>教師となるQ(s_t, a_t)値を、Q学習の式から求める</a></div><div class="lev3 toc-item"><a href="#結合パラメータ更新" data-toc-modified-id="結合パラメータ更新-937"><span class="toc-item-num">9.3.7&nbsp;&nbsp;</span>結合パラメータ更新</a></div><div class="lev4 toc-item"><a href="#ネットワークを訓練モードに切り替える" data-toc-modified-id="ネットワークを訓練モードに切り替える-9371"><span class="toc-item-num">9.3.7.1&nbsp;&nbsp;</span>ネットワークを訓練モードに切り替える</a></div><div class="lev4 toc-item"><a href="#損失関数を計算する（smooth_l1_lossはHuberloss）" data-toc-modified-id="損失関数を計算する（smooth_l1_lossはHuberloss）-9372"><span class="toc-item-num">9.3.7.2&nbsp;&nbsp;</span>損失関数を計算する（smooth_l1_lossはHuberloss）</a></div><div class="lev4 toc-item"><a href="#結合パラメータを更新する" data-toc-modified-id="結合パラメータを更新する-9373"><span class="toc-item-num">9.3.7.3&nbsp;&nbsp;</span>結合パラメータを更新する</a></div><div class="lev1 toc-item"><a href="#sandbox" data-toc-modified-id="sandbox-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>sandbox</a></div><div class="lev3 toc-item"><a href="#経験を記憶するメモリオブジェクトを生成" data-toc-modified-id="経験を記憶するメモリオブジェクトを生成-1001"><span class="toc-item-num">10.0.1&nbsp;&nbsp;</span>経験を記憶するメモリオブジェクトを生成</a></div><div class="lev3 toc-item"><a href="#ニューラルネットワークを構築" data-toc-modified-id="ニューラルネットワークを構築-1002"><span class="toc-item-num">10.0.2&nbsp;&nbsp;</span>ニューラルネットワークを構築</a></div><div class="lev3 toc-item"><a href="#最適化手法の設定" data-toc-modified-id="最適化手法の設定-1003"><span class="toc-item-num">10.0.3&nbsp;&nbsp;</span>最適化手法の設定</a></div><div class="lev3 toc-item"><a href="#replay" data-toc-modified-id="replay-1004"><span class="toc-item-num">10.0.4&nbsp;&nbsp;</span>replay</a></div>


** 5.3、5.4  PyTorchでDQN **

# パッケージのimport

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

# 動画の描画関数の宣言
参考URL http://nbviewer.jupyter.org/github/patrickmineault/xcorr-notebooks/blob/master/Render%20OpenAI%20gym%20as%20GIF.ipynb

In [2]:
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display


def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0),
               dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames),
                                   interval=50)

    anim.save('movie_cartpole_DQN.mp4')  # 動画のファイル名と保存です
    display(display_animation(anim, default_mode='loop'))
    

# namedtuple

本コードでは、namedtupleを使用します。

namedtupleを使うことで、値をフィールド名とペアで格納できます。

すると値に対して、フィールド名でアクセスできて便利です。

https://docs.python.jp/3/library/collections.html#collections.namedtuple

以下は使用例です

In [3]:
from collections import namedtuple

Tr = namedtuple('tr', ('name_a', 'value_b'))
Tr_object = Tr('名前Aです', 100)

print(Tr_object)  # 出力：tr(name_a='名前Aです', value_b=100)
print(Tr_object.value_b)  # 出力：100

tr(name_a='名前Aです', value_b=100)
100


In [4]:
Tr_object.name_a

'名前Aです'

In [5]:
# namedtupleを生成
from collections import namedtuple

Transition = namedtuple(
    'Transition', ('state', 'action', 'next_state', 'reward'))


# 定数の設定

In [6]:
ENV = 'CartPole-v0'  # 使用する課題名
GAMMA = 0.99  # 時間割引率
MAX_STEPS = 200  # 1試行のstep数
NUM_EPISODES = 500  # 最大試行回数

# 経験を保存するメモリクラスを定義します

In [7]:
class ReplayMemory:

    def __init__(self, CAPACITY):
        self.capacity = CAPACITY  # メモリの最大長さ
        self.memory = []  # 経験を保存する変数
        self.index = 0  # 保存するindexを示す変数

    def push(self, state, action, state_next, reward):
        '''transition = (state, action, state_next, reward)をメモリに保存する'''

        if len(self.memory) < self.capacity:
            self.memory.append(None)  # メモリが満タンでないときは足す

        # namedtupleのTransitionを使用し、値とフィールド名をペアにして保存します
        self.memory[self.index] = Transition(state, action, state_next, reward)

        self.index = (self.index + 1) % self.capacity  # 保存するindexを1つずらす

    def sample(self, batch_size):
        '''batch_size分だけ、ランダムに保存内容を取り出す'''
        return random.sample(self.memory, batch_size)

    def __len__(self):
        '''関数lenに対して、現在の変数memoryの長さを返す'''
        return len(self.memory)

In [8]:
import random

## sandbox

In [ ]:
tmp = ReplayMemory(3)

In [ ]:
tmp.push(1,2,3,4)

In [ ]:
tmp.push(2,4,3,1)

In [ ]:
tmp.sample(2)

# Brain

エージェントが持つ脳となるクラスです、DQNを実行します

methodは

* replay: Experience Replayでネットワークの結合パラメータを学習
* decide_action: アクション決定

Q関数をディープラーニングのネットワークをクラスとして定義

In [9]:
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32
CAPACITY = 10000

In [10]:
class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions  # CartPoleの行動（右に左に押す）の2を取得

        # 経験を記憶するメモリオブジェクトを生成
        self.memory = ReplayMemory(CAPACITY)

        # ニューラルネットワークを構築
        self.model = nn.Sequential()
        self.model.add_module('fc1', nn.Linear(num_states, 32))
        self.model.add_module('relu1', nn.ReLU())
        self.model.add_module('fc2', nn.Linear(32, 32))
        self.model.add_module('relu2', nn.ReLU())
        self.model.add_module('fc3', nn.Linear(32, num_actions))

        print(self.model)  # ネットワークの形を出力

        # 最適化手法の設定
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)

    def replay(self):
        '''Experience Replayでネットワークの結合パラメータを学習'''

        # -----------------------------------------
        # 1. メモリサイズの確認
        # -----------------------------------------
        # 1.1 メモリサイズがミニバッチより小さい間は何もしない
        if len(self.memory) < BATCH_SIZE:
            return

        # -----------------------------------------
        # 2. ミニバッチの作成
        # -----------------------------------------
        # 2.1 メモリからミニバッチ分のデータを取り出す
        transitions = self.memory.sample(BATCH_SIZE)

        # 2.2 各変数をミニバッチに対応する形に変形
        # transitionsは1stepごとの(state, action, state_next, reward)が、BATCH_SIZE分格納されている
        # つまり、(state, action, state_next, reward)×BATCH_SIZE
        # これをミニバッチにしたい。つまり
        # (state×BATCH_SIZE, action×BATCH_SIZE, state_next×BATCH_SIZE, reward×BATCH_SIZE)にする
        batch = Transition(*zip(*transitions))

        # 2.3 各変数の要素をミニバッチに対応する形に変形し、ネットワークで扱えるようVariableにする
        # 例えばstateの場合、[torch.FloatTensor of size 1x4]がBATCH_SIZE分並んでいるのですが、
        # それを torch.FloatTensor of size BATCH_SIZEx4 に変換します
        # 状態、行動、報酬、non_finalの状態のミニバッチのVariableを作成
        # catはConcatenates（結合）のことです。
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state
                                           if s is not None])

        # -----------------------------------------
        # 3. 教師信号となるQ(s_t, a_t)値を求める
        # -----------------------------------------
        # 3.1 ネットワークを推論モードに切り替える
        self.model.eval()

        # 3.2 ネットワークが出力したQ(s_t, a_t)を求める
        # self.model(state_batch)は、右左の両方のQ値を出力しており
        # [torch.FloatTensor of size BATCH_SIZEx2]になっている。
        # ここから実行したアクションa_tに対応するQ値を求めるため、action_batchで行った行動a_tが右か左かのindexを求め
        # それに対応するQ値をgatherでひっぱり出す。
        state_action_values = self.model(state_batch).gather(1, action_batch)

        # 3.3 max{Q(s_t+1, a)}値を求める。ただし次の状態があるかに注意。

        # cartpoleがdoneになっておらず、next_stateがあるかをチェックするインデックスマスクを作成
        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None,
                                                    batch.next_state)))
        # まずは全部0にしておく
        next_state_values = torch.zeros(BATCH_SIZE)

        # 次の状態があるindexの最大Q値を求める
        # 出力にアクセスし、max(1)で列方向の最大値の[値、index]を求めます
        # そしてそのQ値（index=0）を出力します
        # detachでその値を取り出します
        next_state_values[non_final_mask] = self.model(
            non_final_next_states).max(1)[0].detach()

        # 3.4 教師となるQ(s_t, a_t)値を、Q学習の式から求める
        expected_state_action_values = reward_batch + GAMMA * next_state_values

        # -----------------------------------------
        # 4. 結合パラメータの更新
        # -----------------------------------------
        # 4.1 ネットワークを訓練モードに切り替える
        self.model.train()

        # 4.2 損失関数を計算する（smooth_l1_lossはHuberloss）
        # expected_state_action_valuesは
        # sizeが[minbatch]になっているので、unsqueezeで[minibatch x 1]へ
        loss = F.smooth_l1_loss(state_action_values,
                                expected_state_action_values.unsqueeze(1))

        # 4.3 結合パラメータを更新する
        self.optimizer.zero_grad()  # 勾配をリセット
        loss.backward()  # バックプロパゲーションを計算
        self.optimizer.step()  # 結合パラメータを更新

    def decide_action(self, state, episode):
        '''現在の状態に応じて、行動を決定する'''
        # ε-greedy法で徐々に最適行動のみを採用する
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            self.model.eval()  # ネットワークを推論モードに切り替える
            with torch.no_grad():
                action = self.model(state).max(1)[1].view(1, 1)
            # ネットワークの出力の最大値のindexを取り出します = max(1)[1]
            # .view(1,1)は[torch.LongTensor of size 1]　を size 1x1 に変換します

        else:
            # 0,1の行動をランダムに返す
            action = torch.LongTensor(
                [[random.randrange(self.num_actions)]])  # 0,1の行動をランダムに返す
            # actionは[torch.LongTensor of size 1x1]の形になります

        return action

## brain 解説（obsolete)


### initialization

```python
class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions  # CartPoleの行動（右に左に押す）の2を取得

        # 経験を記憶するメモリオブジェクトを生成
        self.memory = ReplayMemory(CAPACITY)

        # ニューラルネットワークを構築
        self.model = nn.Sequential()
        self.model.add_module('fc1', nn.Linear(num_states, 32))
        self.model.add_module('relu1', nn.ReLU())
        self.model.add_module('fc2', nn.Linear(32, 32))
        self.model.add_module('relu2', nn.ReLU())
        self.model.add_module('fc3', nn.Linear(32, num_actions))

        print(self.model)  # ネットワークの形を出力

        # 最適化手法の設定
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)
```

### replay

Experience Replayでネットワークの結合パラメータを学習

```python
    def replay(self):
```

#### メモリサイズの確認
メモリサイズがミニバッチより小さい間は何もしない

In [ ]:
%%bash
git commit -a -m "since quite a long time ago"

```python
        if len(self.memory) < BATCH_SIZE:
            return
```

#### ミニバッチの作成
##### メモリからミニバッチ分のデータを取り出す

        transitions = self.memory.sample(BATCH_SIZE)

##### 各変数をミニバッチに対応する形に変形
* transitionsは1stepごとの
$$\text{(state, action, state_next, reward)}$$  
  が、BATCH_SIZE分格納されている
  
* つまり、
  $$\text{(state, action, state_next, reward)}\times\text{BATCH_SIZE}$$
  
* これをミニバッチにしたい。つまり
    $$(\text{state}\times\text{BATCH_SIZE}, ~\text{action}\times\text{BATCH_SIZE}, ~\text{state_next}\times\text{BATCH_SIZE},~ \text{reward}\times\text{BATCH_SIZE})$$
  にする

        batch = Transition(*zip(*transitions))

##### 各変数の要素をミニバッチに対応する形に変形し、ネットワークで扱えるようVariableにする
*  例えばstateの場合、[torch.FloatTensor of size 1x4]がBATCH_SIZE分並んでいる
* それを torch.FloatTensor of size BATCH_SIZEx4 に変換します
* 状態、行動、報酬、non_finalの状態のミニバッチのVariableを作成
*  catはConcatenates（結合）のことです。

```python 
        # 状態、行動、報酬、non_finalの状態のミニバッチのVariableを作成
        # catはConcatenates（結合）のことです。
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state
                                           if s is not None])
```

#### 教師信号となるQ(s_t, a_t)値を求める
##### ネットワークを推論モードに切り替える
        self.model.eval()

##### ネットワークが出力したQ(s_t, a_t)を求める
* self.model(state_batch)は、右左の両方のQ値を出力しており[torch.FloatTensor of size BATCH_SIZEx2]になっている。
* ここから実行したアクションa_tに対応するQ値を求めるため、action_batchで行った行動a_tが右か左かのindexを求め、それに対応するQ値をgatherでひっぱり出す。
        
```python
        state_action_values = self.model(state_batch).gather(1, action_batch)
```

        state_action_values = self.model(state_batch).gather(1, action_batch)

## 解説終わり

In [ ]:
class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions  # CartPoleの行動（右に左に押す）の2を取得


        print(self.model)  # ネットワークの形を出力

        # 
       

    def replay(self):
        '''Experience Replayでネットワークの結合パラメータを学習'''

        # -----------------------------------------
        # 1. メモリサイズの確認
        # -----------------------------------------
        # 1.1 メモリサイズがミニバッチより小さい間は何もしない
        if len(self.memory) < BATCH_SIZE:
            return

        # -----------------------------------------
        # 2. ミニバッチの作成
        # -----------------------------------------
        # 2.1 メモリからミニバッチ分のデータを取り出す
        transitions = self.memory.sample(BATCH_SIZE)

        # 2.2 各変数をミニバッチに対応する形に変形
        # transitionsは1stepごとの(state, action, state_next, reward)が、BATCH_SIZE分格納されている
        # つまり、(state, action, state_next, reward)×BATCH_SIZE
        # これをミニバッチにしたい。つまり
        # (state×BATCH_SIZE, action×BATCH_SIZE, state_next×BATCH_SIZE, reward×BATCH_SIZE)にする
        batch = Transition(*zip(*transitions))

        # 2.3 各変数の要素をミニバッチに対応する形に変形し、ネットワークで扱えるようVariableにする
        # 例えばstateの場合、[torch.FloatTensor of size 1x4]がBATCH_SIZE分並んでいるのですが、
        # それを torch.FloatTensor of size BATCH_SIZEx4 に変換します
        # 状態、行動、報酬、non_finalの状態のミニバッチのVariableを作成
        # catはConcatenates（結合）のことです。
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state
                                           if s is not None])

        # -----------------------------------------
        # 3. 教師信号となるQ(s_t, a_t)値を求める
        # -----------------------------------------
        # 3.1 ネットワークを推論モードに切り替える
        self.model.eval()

        # 3.2 ネットワークが出力したQ(s_t, a_t)を求める
        # self.model(state_batch)は、右左の両方のQ値を出力しており
        # [torch.FloatTensor of size BATCH_SIZEx2]になっている。
        # ここから実行したアクションa_tに対応するQ値を求めるため、action_batchで行った行動a_tが右か左かのindexを求め
        # それに対応するQ値をgatherでひっぱり出す。
        state_action_values = self.model(state_batch).gather(1, action_batch)

        # 3.3 max{Q(s_t+1, a)}値を求める。ただし次の状態があるかに注意。

        # cartpoleがdoneになっておらず、next_stateがあるかをチェックするインデックスマスクを作成
        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None,
                                                    batch.next_state)))
        # まずは全部0にしておく
        next_state_values = torch.zeros(BATCH_SIZE)

        # 次の状態があるindexの最大Q値を求める
        # 出力にアクセスし、max(1)で列方向の最大値の[値、index]を求めます
        # そしてそのQ値（index=0）を出力します
        # detachでその値を取り出します
        next_state_values[non_final_mask] = self.model(
            non_final_next_states).max(1)[0].detach()

        # 3.4 教師となるQ(s_t, a_t)値を、Q学習の式から求める
        expected_state_action_values = reward_batch + GAMMA * next_state_values

        # -----------------------------------------
        # 4. 結合パラメータの更新
        # -----------------------------------------
        # 4.1 ネットワークを訓練モードに切り替える
        self.model.train()

        # 4.2 損失関数を計算する（smooth_l1_lossはHuberloss）
        # expected_state_action_valuesは
        # sizeが[minbatch]になっているので、unsqueezeで[minibatch x 1]へ
        loss = F.smooth_l1_loss(state_action_values,
                                expected_state_action_values.unsqueeze(1))

        # 4.3 結合パラメータを更新する
        self.optimizer.zero_grad()  # 勾配をリセット
        loss.backward()  # バックプロパゲーションを計算
        self.optimizer.step()  # 結合パラメータを更新

    def decide_action(self, state, episode):
        '''現在の状態に応じて、行動を決定する'''
        # ε-greedy法で徐々に最適行動のみを採用する
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            self.model.eval()  # ネットワークを推論モードに切り替える
            with torch.no_grad():
                action = self.model(state).max(1)[1].view(1, 1)
            # ネットワークの出力の最大値のindexを取り出します = max(1)[1]
            # .view(1,1)は[torch.LongTensor of size 1]　を size 1x1 に変換します

        else:
            # 0,1の行動をランダムに返す
            action = torch.LongTensor(
                [[random.randrange(self.num_actions)]])  # 0,1の行動をランダムに返す
            # actionは[torch.LongTensor of size 1x1]の形になります

        return action

In [ ]:
class myBrain0:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions  # CartPoleの行動（右に左に押す）の2を取得

        # 経験を記憶するメモリオブジェクトを生成
        self.memory = ReplayMemory(CAPACITY)

        # ニューラルネットワークを構築
        self.model = nn.Sequential()
        self.model.add_module('fc1', nn.Linear(num_states, 32))
        self.model.add_module('relu1', nn.ReLU())
        self.model.add_module('fc2', nn.Linear(32, 32))
        self.model.add_module('relu2', nn.ReLU())
        self.model.add_module('fc3', nn.Linear(32, num_actions))

        print(self.model)  # ネットワークの形を出力

        # 最適化手法の設定
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)

* メモリサイズの確認
    * メモリサイズがミニバッチより小さい間は何もしない
* ミニバッチの作成
    * メモリからミニバッチ分のデータを取り出す
    

    
```python
        batch = Transition(*zip(*transitions))
```

In [ ]:
1+2 

$1+2$

* 各変数をミニバッチに対応する形に変形
    * transitionsは1stepごとの(state, action, state_next, reward)が、BATCH_SIZE分格納されている
    * つまり$\(state, action, state_next, reward)\times\text{BATCH_SIZE}$    

    * これをミニバッチにしたい。つまり
    $$(\text{state}\times\text{BATCH_SIZE}, \\text{action}\times\text{BATCH_SIZE}, \text{state_next}\times\text{BATCH_SIZE},\text{reward}\times\text{BATCH_SIZE})$$
    にする
   

 * 各変数の要素をミニバッチに対応する形に変形し、ネットワークで扱えるようVariableにする

        例えばstateの場合、[torch.FloatTensor of size 1x4]がBATCH_SIZE分並んでいるのですが、
        それを torch.FloatTensor of size BATCH_SIZEx4 に変換します
        状態、行動、報酬、non_finalの状態のミニバッチのVariableを作成
        catはConcatenates（結合）のことです。

In [ ]:
class myBrain1(myBrain0):
    def replay(self):
        '''Experience Replayでネットワークの結合パラメータを学習'''

        # -----------------------------------------
        # 1. メモリサイズの確認
        # -----------------------------------------
        # 1.1 メモリサイズがミニバッチより小さい間は何もしない
        if len(self.memory) < BATCH_SIZE:
            return

        # -----------------------------------------
        # 2. ミニバッチの作成
        # -----------------------------------------
        # 2.1 メモリからミニバッチ分のデータを取り出す
        transitions = self.memory.sample(BATCH_SIZE)

        # 2.2 各変数をミニバッチに対応する形に変形

        batch = Transition(*zip(*transitions))

        # 2.3 各変数の要素をミニバッチに対応する形に変形し、ネットワークで扱えるようVariableにする
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state
                                           if s is not None])

        # -----------------------------------------
        # 3. 教師信号となるQ(s_t, a_t)値を求める
        # -----------------------------------------
        # 3.1 ネットワークを推論モードに切り替える
        self.model.eval()

        # 3.2 ネットワークが出力したQ(s_t, a_t)を求める

        state_action_values = self.model(state_batch).gather(1, action_batch)

        # 3.3 max{Q(s_t+1, a)}値を求める。ただし次の状態があるかに注意。

        # cartpoleがdoneになっておらず、next_stateがあるかをチェックするインデックスマスクを作成
        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None,
                                                    batch.next_state)))
        # まずは全部0にしておく
        next_state_values = torch.zeros(BATCH_SIZE)

        # 次の状態があるindexの最大Q値を求める
        # 出力にアクセスし、max(1)で列方向の最大値の[値、index]を求めます
        # そしてそのQ値（index=0）を出力します
        # detachでその値を取り出します
        next_state_values[non_final_mask] = self.model(
            non_final_next_states).max(1)[0].detach()

        # 3.4 教師となるQ(s_t, a_t)値を、Q学習の式から求める
        expected_state_action_values = reward_batch + GAMMA * next_state_values

        # -----------------------------------------
        # 4. 結合パラメータの更新
        # -----------------------------------------
        # 4.1 ネットワークを訓練モードに切り替える
        self.model.train()

        # 4.2 損失関数を計算する（smooth_l1_lossはHuberloss）
        # expected_state_action_valuesは
        # sizeが[minbatch]になっているので、unsqueezeで[minibatch x 1]へ
        loss = F.smooth_l1_loss(state_action_values,
                                expected_state_action_values.unsqueeze(1))

        # 4.3 結合パラメータを更新する
        self.optimizer.zero_grad()  # 勾配をリセット
        loss.backward()  # バックプロパゲーションを計算
        self.optimizer.step()  # 結合パラメータを更新

In [ ]:
class myBrain2(myBrain1):
    def decide_action(self, state, episode):
        '''現在の状態に応じて、行動を決定する'''
        # 
        -greedy法で徐々に最適行動のみを採用する
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            self.model.eval()  # ネットワークを推論モードに切り替える
            with torch.no_grad():
                action = self.model(state).max(1)[1].view(1, 1)
            # ネットワークの出力の最大値のindexを取り出します = max(1)[1]
            # .view(1,1)は[torch.LongTensor of size 1]　を size 1x1 に変換します

        else:
            # 0,1の行動をランダムに返す
            action = torch.LongTensor(
                [[random.randrange(self.num_actions)]])  # 0,1の行動をランダムに返す
            # actionは[torch.LongTensor of size 1x1]の形になります

        return action

# Agent
CartPoleで動くエージェントクラスです、棒付き台車そのものになります

methodは


* 行動価値Qを更新
*  状態を与えると行動を決定
*  状態、選択するアクション、次の状態、報酬などを記憶

In [11]:
class Agent:
    def __init__(self, num_states, num_actions):
        '''課題の状態と行動の数を設定する'''
        self.brain = Brain(num_states, num_actions)  # エージェントが行動を決定するための頭脳を生成

    def update_q_function(self):
        '''Q関数を更新する'''
        self.brain.replay()

    def get_action(self, state, episode):
        '''行動を決定する'''
        action = self.brain.decide_action(state, episode)
        return action

    def memorize(self, state, action, state_next, reward):
        '''memoryオブジェクトに、state, action, state_next, rewardの内容を保存する'''
        self.brain.memory.push(state, action, state_next, reward)

# Environment
CartPoleを実行する環境のクラスです

* for loopは二重になっている
* 外側のループはEPISODEに関して
    * Episode = 試行：一回ポールを立てて倒れるか200ステップ経過するまでを１エピソードと数える
    * 内側のループはステップに関して
        * 初期状態のBrainを使って、1ステップ目から左右にコントロールしていくことからスタート
        * 各ステップごとに状態と遷移を記録する。
        * 同様に各ステップごとに行動価値関数をアップデートしていく
    * 10エピソード連続で200ステップまで持ちこたえられたら成功
    

## original

In [12]:
class Environment:

    def __init__(self):
        self.env = gym.make(ENV)  # 実行する課題を設定
        num_states = self.env.observation_space.shape[0]  # 課題の状態数4を取得
        num_actions = self.env.action_space.n  # CartPoleの行動（右に左に押す）の2を取得
        self.agent = Agent(num_states, num_actions)  # 環境内で行動するAgentを生成

        
    def run(self):
        '''実行'''
        episode_10_list = np.zeros(10)  # 10試行分の立ち続けたstep数を格納し、平均ステップ数を出力に利用
        complete_episodes = 0  # 195step以上連続で立ち続けた試行数
        episode_final = False  # 最後の試行フラグ
        frames = []  # 最後の試行を動画にするために画像を格納する変数

        for episode in range(NUM_EPISODES):  # 最大試行数分繰り返す
            observation = self.env.reset()  # 環境の初期化

            state = observation  # 観測をそのまま状態sとして使用
            state = torch.from_numpy(state).type(
                torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
            state = torch.unsqueeze(state, 0)  # size 4をsize 1x4に変換

            for step in range(MAX_STEPS):  # 1エピソードのループ

                if episode_final is True:  # 最終試行ではframesに各時刻の画像を追加していく
                    frames.append(self.env.render(mode='rgb_array'))

                action = self.agent.get_action(state, episode)  # 行動を求める

                # 行動a_tの実行により、s_{t+1}とdoneフラグを求める
                # actionから.item()を指定して、中身を取り出す
                observation_next, _, done, _ = self.env.step(
                    action.item())  # rewardとinfoは使わないので_にする

                # 報酬を与える。さらにepisodeの終了評価と、state_nextを設定する
                if done:  # ステップ数が200経過するか、一定角度以上傾くとdoneはtrueになる
                    state_next = None  # 次の状態はないので、Noneを格納

                    # 直近10episodeの立てたstep数リストに追加
                    episode_10_list = np.hstack(
                        (episode_10_list[1:], step + 1))

                    if step < 195:
                        reward = torch.FloatTensor(
                            [-1.0])  # 途中でこけたら罰則として報酬-1を与える
                        complete_episodes = 0  # 連続成功記録をリセット
                    else:
                        reward = torch.FloatTensor([1.0])  # 立ったまま終了時は報酬1を与える
                        complete_episodes = complete_episodes + 1  # 連続記録を更新
                else:
                    reward = torch.FloatTensor([0.0])  # 普段は報酬0
                    state_next = observation_next  # 観測をそのまま状態とする
                    state_next = torch.from_numpy(state_next).type(
                        torch.FloatTensor)  # numpy変数をPyTorchのテンソルに変換
                    state_next = torch.unsqueeze(state_next, 0)  # size 4をsize 1x4に変換

                # メモリに経験を追加
                self.agent.memorize(state, action, state_next, reward)

                # Experience ReplayでQ関数を更新する
                self.agent.update_q_function()

                # 観測の更新
                state = state_next

                # 終了時の処理
                if done:
                    print('%d Episode: Finished after %d steps：10試行の平均step数 = %.1lf' % (
                        episode, step + 1, episode_10_list.mean()))
                    break

            if episode_final is True:
                # 動画を保存と描画
                display_frames_as_gif(frames)
                break

            # 10連続で200step経ち続けたら成功
            if complete_episodes >= 10:
                print('10回連続成功')
                episode_final = True  # 次の試行を描画を行う最終試行とする

In [13]:
# main クラス
cartpole_env = Environment()
cartpole_env.run()


Sequential(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)
0 Episode: Finished after 14 steps：10試行の平均step数 = 1.4
1 Episode: Finished after 11 steps：10試行の平均step数 = 2.5
2 Episode: Finished after 9 steps：10試行の平均step数 = 3.4
3 Episode: Finished after 9 steps：10試行の平均step数 = 4.3
4 Episode: Finished after 9 steps：10試行の平均step数 = 5.2
5 Episode: Finished after 8 steps：10試行の平均step数 = 6.0
6 Episode: Finished after 8 steps：10試行の平均step数 = 6.8
7 Episode: Finished after 11 steps：10試行の平均step数 = 7.9
8 Episode: Finished after 10 steps：10試行の平均step数 = 8.9
9 Episode: Finished after 12 steps：10試行の平均step数 = 10.1
10 Episode: Finished after 12 steps：10試行の平均step数 = 9.9
11 Episode: Finished after 12 steps：10試行の平均step数 = 10.0
12 Episode: Finished after 14 steps：10試行の平均step数 = 10.5
13 Episode: Finished after 9 steps：10試行の平均step数 = 10.5
14 Episode: Fi

145 Episode: Finished after 121 steps：10試行の平均step数 = 92.9
146 Episode: Finished after 188 steps：10試行の平均step数 = 104.9
147 Episode: Finished after 58 steps：10試行の平均step数 = 100.5
148 Episode: Finished after 200 steps：10試行の平均step数 = 108.9
149 Episode: Finished after 148 steps：10試行の平均step数 = 117.3
150 Episode: Finished after 149 steps：10試行の平均step数 = 121.1
151 Episode: Finished after 64 steps：10試行の平均step数 = 121.0
152 Episode: Finished after 89 steps：10試行の平均step数 = 117.7
153 Episode: Finished after 98 steps：10試行の平均step数 = 118.3
154 Episode: Finished after 200 steps：10試行の平均step数 = 131.5
155 Episode: Finished after 185 steps：10試行の平均step数 = 137.9
156 Episode: Finished after 193 steps：10試行の平均step数 = 138.4
157 Episode: Finished after 174 steps：10試行の平均step数 = 150.0
158 Episode: Finished after 182 steps：10試行の平均step数 = 148.2
159 Episode: Finished after 200 steps：10試行の平均step数 = 153.4
160 Episode: Finished after 200 steps：10試行の平均step数 = 158.5
161 Episode: Finished after 200 steps：10試行の平均step数 = 172.1
16

NoSuchDisplayException: Cannot connect to "None"

## 多少改造したenvironment

In [28]:
class myEnvironment:

    def __init__(self):
        self.env = gym.make(ENV)  # 実行する課題を設定
        num_states = self.env.observation_space.shape[0]  # 課題の状態数4を取得
        num_actions = self.env.action_space.n  # CartPoleの行動（右に左に押す）の2を取得
        self.agent = Agent(num_states, num_actions)  # 環境内で行動するAgentを生成
        self.frames = []

        
    def run(self):
        '''実行'''
        episode_10_list = np.zeros(10)  # 10試行分の立ち続けたstep数を格納し、平均ステップ数を出力に利用
        complete_episodes = 0  # 195step以上連続で立ち続けた試行数
        episode_final = False  # 最後の試行フラグ
        self.frames = []  # 最後の試行を動画にするために画像を格納する変数

        for episode in range(NUM_EPISODES):  # 最大試行数分繰り返す
            observation = self.env.reset()  # 環境の初期化

            state = observation  # 観測をそのまま状態sとして使用
            state = torch.from_numpy(state).type(
                torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
            state = torch.unsqueeze(state, 0)  # size 4をsize 1x4に変換

            for step in range(MAX_STEPS):  # 1エピソードのループ

                if episode_final is True:  # 最終試行ではframesに各時刻の画像を追加していく
                    self.frames.append(self.env.render(mode='rgb_array'))

                action = self.agent.get_action(state, episode)  # 行動を求める

                # 行動a_tの実行により、s_{t+1}とdoneフラグを求める
                # actionから.item()を指定して、中身を取り出す
                observation_next, _, done, _ = self.env.step(
                    action.item())  # rewardとinfoは使わないので_にする

                # 報酬を与える。さらにepisodeの終了評価と、state_nextを設定する
                if done:  # ステップ数が200経過するか、一定角度以上傾くとdoneはtrueになる
                    state_next = None  # 次の状態はないので、Noneを格納

                    # 直近10episodeの立てたstep数リストに追加
                    episode_10_list = np.hstack(
                        (episode_10_list[1:], step + 1))

                    if step < 195:
                        reward = torch.FloatTensor(
                            [-1.0])  # 途中でこけたら罰則として報酬-1を与える
                        complete_episodes = 0  # 連続成功記録をリセット
                    else:
                        reward = torch.FloatTensor([1.0])  # 立ったまま終了時は報酬1を与える
                        complete_episodes = complete_episodes + 1  # 連続記録を更新
                else:
                    reward = torch.FloatTensor([0.0])  # 普段は報酬0
                    state_next = observation_next  # 観測をそのまま状態とする
                    state_next = torch.from_numpy(state_next).type(
                        torch.FloatTensor)  # numpy変数をPyTorchのテンソルに変換
                    state_next = torch.unsqueeze(state_next, 0)  # size 4をsize 1x4に変換

                # メモリに経験を追加
                self.agent.memorize(state, action, state_next, reward)

                # Experience ReplayでQ関数を更新する
                self.agent.update_q_function()

                # 観測の更新
                state = state_next

                # 終了時の処理
                if done:
                    print('%d Episode: Finished after %d steps：10試行の平均step数 = %.1lf' % (
                        episode, step + 1, episode_10_list.mean()))
                    break

            if episode_final is True:
                # 動画を保存と描画
                print("length of the final frame is {}".format(len(frames)))
                display_frames_as_gif(self.frames)
                break

            # 10連続で200step経ち続けたら成功
            if complete_episodes >= 10:
                print('10回連続成功')
                episode_final = True  # 次の試行を描画を行う最終試行とする

In [29]:
# main クラス
cartpole_env = myEnvironment()

Sequential(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)


In [30]:
cartpole_env.run()

0 Episode: Finished after 12 steps：10試行の平均step数 = 1.2
1 Episode: Finished after 10 steps：10試行の平均step数 = 2.2
2 Episode: Finished after 12 steps：10試行の平均step数 = 3.4
3 Episode: Finished after 9 steps：10試行の平均step数 = 4.3
4 Episode: Finished after 10 steps：10試行の平均step数 = 5.3
5 Episode: Finished after 9 steps：10試行の平均step数 = 6.2
6 Episode: Finished after 9 steps：10試行の平均step数 = 7.1
7 Episode: Finished after 10 steps：10試行の平均step数 = 8.1
8 Episode: Finished after 9 steps：10試行の平均step数 = 9.0
9 Episode: Finished after 8 steps：10試行の平均step数 = 9.8
10 Episode: Finished after 10 steps：10試行の平均step数 = 9.6
11 Episode: Finished after 10 steps：10試行の平均step数 = 9.6
12 Episode: Finished after 11 steps：10試行の平均step数 = 9.5
13 Episode: Finished after 10 steps：10試行の平均step数 = 9.6
14 Episode: Finished after 9 steps：10試行の平均step数 = 9.5
15 Episode: Finished after 8 steps：10試行の平均step数 = 9.4
16 Episode: Finished after 10 steps：10試行の平均step数 = 9.5
17 Episode: Finished after 10 steps：10試行の平均step数 = 9.5
18 Episode: Finished after 

145 Episode: Finished after 191 steps：10試行の平均step数 = 193.7
146 Episode: Finished after 195 steps：10試行の平均step数 = 194.0
147 Episode: Finished after 178 steps：10試行の平均step数 = 192.6
148 Episode: Finished after 200 steps：10試行の平均step数 = 193.1
149 Episode: Finished after 200 steps：10試行の平均step数 = 193.1
150 Episode: Finished after 197 steps：10試行の平均step数 = 192.8
151 Episode: Finished after 185 steps：10試行の平均step数 = 191.3
152 Episode: Finished after 200 steps：10試行の平均step数 = 191.3
153 Episode: Finished after 188 steps：10試行の平均step数 = 192.3
154 Episode: Finished after 200 steps：10試行の平均step数 = 193.4
155 Episode: Finished after 200 steps：10試行の平均step数 = 194.3
156 Episode: Finished after 200 steps：10試行の平均step数 = 194.8
157 Episode: Finished after 200 steps：10試行の平均step数 = 197.0
158 Episode: Finished after 200 steps：10試行の平均step数 = 197.0
159 Episode: Finished after 200 steps：10試行の平均step数 = 197.0
160 Episode: Finished after 200 steps：10試行の平均step数 = 197.3
161 Episode: Finished after 200 steps：10試行の平均step数 = 198

NameError: name 'base' is not defined

In [31]:
cartpole_env.env.render(mode='rgb_array')

NameError: name 'base' is not defined

# classでなくしてみる

## envのinit

In [ ]:
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32
CAPACITY = 10000

In [ ]:
my_env = gym.make(ENV)  # 実行する課題を設定

In [ ]:
num_states = my_env.observation_space.shape[0]  # 課題の状態数4を取得
num_actions = my_env.action_space.n  # CartPoleの行動（右に左に押す）の2を取得

### Agent作成

In [ ]:
my_agent = Agent(num_states, num_actions)  # 環境内で行動するAgentを生成

In [ ]:
my_env.observation_space

In [ ]:
my_env.action_space

In [ ]:
episode_10_list = np.zeros(10)  # 10試行分の立ち続けたstep数を格納し、平均ステップ数を出力に利用
complete_episodes = 0  # 195step以上連続で立ち続けた試行数
episode_final = False  # 最後の試行フラグ
frames = []  # 最後の試行を動画にするために画像を格納する変数


## envのfor loop

In [ ]:
episode = 0

In [ ]:
observation = my_env.reset()  # 環境の初期化

In [ ]:
state = observation  # 観測をそのまま状態sとして使用
state = torch.from_numpy(state).type(
    torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
state = torch.unsqueeze(state, 0)  # size 4をsize 1x4に変換


In [ ]:
state

In [ ]:
type(state)

In [ ]:
#for step in range(MAX_STEPS):  # 1エピソードのループ
step = 0

In [ ]:
if episode_final is True:  # 最終試行ではframesに各時刻の画像を追加していく
    frames.append(self.env.render(mode='rgb_array'))

### agent.get_actionは$\epsilon$-greedy法により決定されたaction(この例の場合、右か左かのゼロイチ）を返す

In [ ]:
action = my_agent.get_action(state, episode)  # 行動を求める

In [ ]:
type(action)

In [ ]:
action.item()

### actionを与えるとenvがどうなるかを計算してくれる：

In [ ]:
# 行動a_tの実行により、s_{t+1}とdoneフラグを求める
# actionから.item()を指定して、中身を取り出す
observation_next, _, done, _ = my_env.step(
    action.item())  # rewardとinfoは使わないので_にする

In [ ]:
done

### 終了していたら

* episode_10_list の先頭を削除して、末尾に「今回は何ステップ続いたか」を追加
* 195ステップ未満だった場合、報酬-1を付与
* 195ステップ以上だったら報酬＋１を付与

In [ ]:
# 報酬を与える。さらにepisodeの終了評価と、state_nextを設定する
if done:  # ステップ数が200経過するか、一定角度以上傾くとdoneはtrueになる
    state_next = None  # 次の状態はないので、Noneを格納

    # 直近10episodeの立てたstep数リストに追加
    episode_10_list = np.hstack(
        (episode_10_list[1:], step + 1))

    if step < 195:
        reward = torch.FloatTensor(
            [-1.0])  # 途中でこけたら罰則として報酬-1を与える
        complete_episodes = 0  # 連続成功記録をリセット
    else:
        reward = torch.FloatTensor([1.0])  # 立ったまま終了時は報酬1を与える
        complete_episodes = complete_episodes + 1  # 連続記録を更新

In [ ]:
if not done:
    reward = torch.FloatTensor([0.0])  # 普段は報酬0
    state_next = observation_next  # 観測をそのまま状態とする
    state_next = torch.from_numpy(state_next).type(
        torch.FloatTensor)  # numpy変数をPyTorchのテンソルに変換
    state_next = torch.unsqueeze(state_next, 0)  # size 4をsize 1x4に変換

### エージェントが（状態、アクション、次のアクション、報酬）を記憶

In [ ]:
# メモリに経験を追加
my_agent.memorize(state, action, state_next, reward)

### Experience ReplayでQ関数を更新する

In [ ]:
my_agent.update_q_function()

In [ ]:
# 観測の更新
state = state_next

In [ ]:
# 終了時の処理
if done:
    print('%d Episode: Finished after %d steps：10試行の平均step数 = %.1lf' % (
        episode, step + 1, episode_10_list.mean()))
    break

## brain

### サンプルがたまった学習済みのbrainを拝借

In [ ]:
my_brain = cartpole_env.agent.brain

### mini batch取得

In [ ]:
transitions = my_brain.memory.sample(BATCH_SIZE)

In [ ]:
len(transitions)

In [ ]:
transitions[0:3]

### mini batchをtorchが食える形式に変形

##### 各変数をミニバッチに対応する形に変形
* transitionsは1stepごとの
$$\text{(state, action, state_next, reward)}$$  
  が、BATCH_SIZE分格納されている
  
* つまり、
  $$\text{(state, action, state_next, reward)}\times\text{BATCH_SIZE}$$
  
* これをミニバッチにしたい。つまり
    $$(\text{state}\times\text{BATCH_SIZE}, ~\text{action}\times\text{BATCH_SIZE}, ~\text{state_next}\times\text{BATCH_SIZE},~ \text{reward}\times\text{BATCH_SIZE})$$
  にする

In [14]:
batch = Transition(*zip(*transitions))

NameError: name 'transitions' is not defined

In [ ]:
len(batch)

In [ ]:
batch[0][0:3]

In [ ]:
batch.state[0:3]

##### 各変数の要素をミニバッチに対応する形に変形し、ネットワークで扱えるようVariableにする
*  例えばstateの場合、[torch.FloatTensor of size 1x4]がBATCH_SIZE分並んでいる
* それを torch.FloatTensor of size BATCH_SIZEx4 に変換します
* 状態、行動、報酬、non_finalの状態のミニバッチのVariableを作成
*  catはConcatenates（結合）のことです。

In [ ]:
state_batch = torch.cat(batch.state)
action_batch = torch.cat(batch.action)
reward_batch = torch.cat(batch.reward)
non_final_next_states = torch.cat([s for s in batch.next_state
                                   if s is not None])

In [ ]:
state_batch.shape


### 推論モードへ切り替え

In [ ]:
# -----------------------------------------
# 3. 教師信号となるQ(s_t, a_t)値を求める
# -----------------------------------------
# 3.1 ネットワークを推論モードに切り替える
my_brain.model.eval()

###  ネットワークが出力したQ(s_t, a_t)を求める (state_action_valueに格納）

In [ ]:
# self.model(state_batch)は、右左の両方のQ値を出力しており
# [torch.FloatTensor of size BATCH_SIZEx2]になっている。
# ここから実行したアクションa_tに対応するQ値を求めるため、action_batchで行った行動a_tが右か左かのindexを求め
# それに対応するQ値をgatherでひっぱり出す。
state_action_values = my_brain.model(state_batch).gather(1, action_batch)

In [ ]:
state_action_values.shape

In [ ]:
state_action_values

### 教師となる行動価値関数（expected_state_action_values）を計算
cartpoleがdoneになっておらず、next_stateがあるかをチェックするインデックスマスクを作成

In [ ]:
non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None,
                                            batch.next_state)))

In [ ]:
non_final_mask

In [ ]:
# まずは全部0にしておく
next_state_values = torch.zeros(BATCH_SIZE)

#### doneでないnext state:

In [ ]:
non_final_next_states[0:10]

#### next statenにおける行動価値関数

In [ ]:
my_brain.model(non_final_next_states)[0:10]

#### next stateにおける行動価値関数の最大値

In [ ]:
my_brain.model(non_final_next_states).max(1)

このように最大値と最大値を返すインデックスが返る    

In [ ]:
type(my_brain.model(non_final_next_states).max(1))

In [ ]:
# 次の状態があるindexの最大Q値を求める
# 出力にアクセスし、max(1)で列方向の最大値の[値、index]を求めます
# そしてそのQ値（index=0）を出力します
# detachでその値を取り出します
next_state_values[non_final_mask] = my_brain.model(
    non_final_next_states).max(1)[0].detach()

In [ ]:
next_state_values

$\uparrow$ next stateがdoneでないsampleに関してはnext stateにおける行動価値関数のmax(最適行動をとったときの値)が、next stateがdoneならゼロが格納されている

In [ ]:
%%bash
git commit -a -m "checkpoint"

#### 教師となるQ(s_t, a_t)値を、Q学習の式から求める

In [ ]:
expected_state_action_values = reward_batch + GAMMA * next_state_values

### 結合パラメータ更新

#### ネットワークを訓練モードに切り替える

In [ ]:
my_brain.model.train()

#### 損失関数を計算する（smooth_l1_lossはHuberloss）
expected_state_action_valuesは
sizeが[minbatch]になっているので、unsqueezeで[minibatch x 1]へ

In [ ]:
loss = F.smooth_l1_loss(state_action_values,
                        expected_state_action_values.unsqueeze(1))

In [ ]:
type(state_action_values)

In [ ]:
state_action_values.

In [ ]:
type(expected_state_action_values)

In [ ]:
loss

#### 結合パラメータを更新する

In [ ]:
my_brain.optimizer.zero_grad()  # 勾配をリセット

In [ ]:
loss.backward()  # バックプロパゲーションを計算

optimizerはlearning rateを指定済みのAdam

In [ ]:
my_brain.optimizer.step()  # 結合パラメータを更新

# sandbox

In [ ]:
print(my_brain.model.parameters())

### 経験を記憶するメモリオブジェクトを生成

In [ ]:
my_memory = ReplayMemory(CAPACITY)

In [ ]:
%%bash
git commit -a -m "checkpoint"

### ニューラルネットワークを構築

In [ ]:
my_model = nn.Sequential()
my_model.add_module('fc1', nn.Linear(num_states, 32))
my_model.add_module('relu1', nn.ReLU())
my_model.add_module('fc2', nn.Linear(32, 32))
my_model.add_module('relu2', nn.ReLU())
my_model.add_module('fc3', nn.Linear(32, num_actions))

### 最適化手法の設定

In [ ]:
my_optimizer = optim.Adam(my_model.parameters(), lr=0.0001)

### replay